```
bryan@ichabod mac_bucket % for file in *_ip.json; do
  databricks fs cp "$file" dbfs:/Volumes/geodata/petra/ip_raw/
done
```


In [ ]:
df = spark.read.format("json").load("/Volumes/geodata/petra/ip_raw/")

display(df)

In [ ]:
from common.transforms import string_to_iso_date, generate_hash, replace_10e30_with_null
import pyspark.sql.functions as F

df_flat = df.select(
    F.col("repo_id"),
    F.col("uwi.uwi").alias("uwi"),
    F.col("uwi.wsn").alias("wsn"),
    F.col("pdtest.base").alias("base"),
    F.col("pdtest.bhp").alias("bhp"),
    F.col("pdtest.bht").alias("bht"),
    F.col("pdtest.caof").alias("caof"),
    F.col("pdtest.chgdate").alias("app_row_changed"),
    F.col("pdtest.choke").alias("choke"),
    F.col("pdtest.date").alias("date"),
    F.col("pdtest.duration").alias("duration"),
    F.col("pdtest.fcp").alias("fcp"),
    F.col("pdtest.fmname").alias("fmname"),
    F.col("pdtest.ftp").alias("ftp"),
    F.col("pdtest.gasgty").alias("gasgty"),
    F.col("pdtest.gasunit").alias("gasunit"),
    F.col("pdtest.gasvol").alias("gasvol"),
    F.col("pdtest.gor").alias("gor"),
    F.col("pdtest.numtreat").alias("numtreat"),
    F.col("pdtest.oilgty").alias("oilgty"),
    F.col("pdtest.oilunit").alias("oilunit"),
    F.col("pdtest.oilvol").alias("oilvol"),
    F.col("pdtest.recid").alias("recid"),
    F.col("pdtest.remark").alias("remark"),
    F.col("pdtest.scp").alias("scp"),
    F.col("pdtest.stp").alias("stp"),
    F.col("pdtest.testtype").alias("testtype"),
    F.col("pdtest.top").alias("top"),
    F.col("pdtest.treat").alias("treat"),
    F.col("pdtest.unitstype").alias("unitstype"),
    F.col("pdtest.wtrunit").alias("wtrunit"),
    F.col("pdtest.wtrvol").alias("wtrvol"),
)


df_ip = df_flat


# enforce timestamp for dates
for col_name in ["app_row_changed", "date"]:
    df_ip = string_to_iso_date(df_ip, col_name, col_name)


# ensure real nulls
for col_name in [
    "base",
    "bhp",
    "bht",
    "caof",
    "choke",
    "duration",
    "fcp",
    "ftp",
    "gasgty",
    "gasvol",
    "gor",
    "oilgty",
    "oilvol",
    "scp",
    "stp",
    "top",
    "wtrvol",
]:
    df_ip = replace_10e30_with_null(df_ip, col_name, col_name)


# add id hash
id_columns = ["repo_id", "uwi"]
df_ip = generate_hash(df_ip, "id", "ip", *id_columns)


display(df_ip)

In [ ]:
from common.transforms import upsert_dataframe_to_table

result = upsert_dataframe_to_table(df_ip, "geodata.petra.ip_bronze")
display(result)